## Calculate approximate distances from boulders and LM to geophones

### Import libraries

In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
import os
import math
import datetime

# Import functions
import sys
fxndir = '../functions/'
sys.path.insert(0,fxndir)
from moon2data import *

### Load catalog of geophone coordinates

In [2]:
mqdir1 = '../catalogs/coordinates/'
coord_cat = pd.read_csv(mqdir1 + 'Apollo17_Cartesian_Coordinates.csv')

### Unpack coordinates of geophones

In [3]:
# Geophone 1
geo1_row = coord_cat.loc[coord_cat.Instrument == 'geo1'].iloc[0]
x_geo1 = geo1_row.x_m
y_geo1 = geo1_row.y_m

# Geophone 2
geo2_row = coord_cat.loc[coord_cat.Instrument == 'geo2'].iloc[0]
x_geo2 = geo2_row.x_m
y_geo2 = geo2_row.y_m

# Geophone 3
geo3_row = coord_cat.loc[coord_cat.Instrument == 'geo3'].iloc[0]
x_geo3 = geo3_row.x_m
y_geo3 = geo3_row.y_m

# Geophone 4
geo4_row = coord_cat.loc[coord_cat.Instrument == 'geo4'].iloc[0]
x_geo4 = geo4_row.x_m
y_geo4 = geo4_row.y_m

## Calculate distances from boulders to geophones

### Load approximate coordinates of boulders

Estimated from Haase et al. (2019)

In [4]:
boulder_cat = pd.read_csv(mqdir1 + 'Apollo17_boulder_coordinates.csv')
boulder_cat.drop(list(boulder_cat.filter(regex='Unnamed|index')), axis=1, inplace=True)

### Calculate distances 

In [5]:
geo1_dists = []
geo2_dists = []
geo3_dists = []
geo4_dists = []
for r in np.arange(0,len(boulder_cat)):
    
    # Unpack coordinates
    row = boulder_cat.iloc[r]
    x_boulder = row.x_m
    y_boulder = row.y_m
    
    # Calculate distance
    # Geophone 1
    geo1_dist = np.sqrt((x_boulder - x_geo1)**2 + (y_boulder - y_geo1)**2)
    geo1_dists.append(geo1_dist)
    
    # Geophone 2
    geo2_dist = np.sqrt((x_boulder - x_geo2)**2 + (y_boulder - y_geo2)**2)
    geo2_dists.append(geo2_dist)
    
    # Geophone 3
    geo3_dist = np.sqrt((x_boulder - x_geo3)**2 + (y_boulder - y_geo3)**2)
    geo3_dists.append(geo3_dist)
    
    # Geophone 4
    geo4_dist = np.sqrt((x_boulder - x_geo4)**2 + (y_boulder - y_geo4)**2)
    geo4_dists.append(geo4_dist)

In [6]:
boulder_cat['geo1_xy_dist_m'] = geo1_dists
boulder_cat['geo2_xy_dist_m'] = geo2_dists
boulder_cat['geo3_xy_dist_m'] = geo3_dists
boulder_cat['geo4_xy_dist_m'] = geo4_dists
boulder_cat = boulder_cat[['x_m', 'y_m', 'diameter_m', 'name', 'geo1_xy_dist_m', 'geo2_xy_dist_m',
       'geo3_xy_dist_m', 'geo4_xy_dist_m']]
boulder_cat.to_csv(mqdir1 + 'Apollo17_boulder_coordinates.csv',index=False)

## Build a catalog of source-receiver distances

### Initialize arrays

In [7]:
sources = ['LM']
dist_geo1_LM = geo1_row.XYZ_dist_LM_m
geo1_dists = [dist_geo1_LM]
dist_geo2_LM = geo2_row.XYZ_dist_LM_m
geo2_dists = [dist_geo2_LM]
dist_geo3_LM = geo3_row.XYZ_dist_LM_m
geo3_dists = [dist_geo3_LM]
dist_geo4_LM = geo4_row.XYZ_dist_LM_m
geo4_dists = [dist_geo4_LM]

### Load coordinates of boulders

In [8]:
# Geophone Rock
boulder_cat = pd.read_csv(mqdir1 + 'Apollo17_boulder_coordinates.csv')
row_georock = boulder_cat.loc[boulder_cat.name == 'Geophone rock'].iloc[0]
x_georock = row_georock.x_m
y_georock = row_georock.y_m

# R2
row_R2 = boulder_cat.loc[boulder_cat.name == 'R2'].iloc[0]
x_R2 = row_R2.x_m
y_R2 = row_R2.y_m

# Geophone 4 Rock 1
row_rock1 = boulder_cat.loc[boulder_cat.name == 'GEO 4 rock 1'].iloc[0]
x_rock1 = row_rock1.x_m
y_rock1 = row_rock1.y_m

# Geophone 4 Rock 2
row_rock2 = boulder_cat.loc[boulder_cat.name == 'GEO 4 rock 2'].iloc[0]
x_rock2 = row_rock2.x_m
y_rock2 = row_rock2.y_m

# Geophone 4 Rock 3
row_rock3 = boulder_cat.loc[boulder_cat.name == 'GEO 4 rock 3'].iloc[0]
x_rock3 = row_rock3.x_m
y_rock3 = row_rock3.y_m

### Calculate distances from geophones to boulders

In [9]:
# Geophone Rock
sources.append('Geophone rock')
dist_geo1_georock = np.sqrt((x_geo1 - x_georock)**2 + (y_geo1 - y_georock)**2)
geo1_dists.append(dist_geo1_georock)
dist_geo2_georock = np.sqrt((x_geo2 - x_georock)**2 + (y_geo2 - y_georock)**2)
geo2_dists.append(dist_geo2_georock)
dist_geo3_georock = np.sqrt((x_geo3 - x_georock)**2 + (y_geo3 - y_georock)**2)
geo3_dists.append(dist_geo3_georock)
dist_geo4_georock = np.sqrt((x_geo4 - x_georock)**2 + (y_geo4 - y_georock)**2)
geo4_dists.append(dist_geo4_georock)

# R2
sources.append('R2')
dist_geo1_R2 = np.sqrt((x_geo1 - x_R2)**2 + (y_geo1 - y_R2)**2)
geo1_dists.append(dist_geo1_R2)
dist_geo2_R2 = np.sqrt((x_geo2 - x_R2)**2 + (y_geo2 - y_R2)**2)
geo2_dists.append(dist_geo2_R2)
dist_geo3_R2 = np.sqrt((x_geo3 - x_R2)**2 + (y_geo3 - y_R2)**2)
geo3_dists.append(dist_geo3_R2)
dist_geo4_R2 = np.sqrt((x_geo4 - x_R2)**2 + (y_geo4 - y_R2)**2)
geo4_dists.append(dist_geo4_R2)

# Geophone 4 Rock 1
sources.append('GEO 4 rock 1')
dist_geo1_rock1 = np.sqrt((x_geo1 - x_rock1)**2 + (y_geo1 - y_rock1)**2)
geo1_dists.append(dist_geo1_rock1)
dist_geo2_rock1 = np.sqrt((x_geo2 - x_rock1)**2 + (y_geo2 - y_rock1)**2)
geo2_dists.append(dist_geo2_rock1)
dist_geo3_rock1 = np.sqrt((x_geo3 - x_rock1)**2 + (y_geo3 - y_rock1)**2)
geo3_dists.append(dist_geo3_rock1)
dist_geo4_rock1 = np.sqrt((x_geo4 - x_rock1)**2 + (y_geo4 - y_rock1)**2)
geo4_dists.append(dist_geo4_rock1)

# Geophone 4 Rock 2
sources.append('GEO 4 rock 2')
dist_geo1_rock2 = np.sqrt((x_geo1 - x_rock2)**2 + (y_geo1 - y_rock2)**2)
geo1_dists.append(dist_geo1_rock2)
dist_geo2_rock2 = np.sqrt((x_geo2 - x_rock2)**2 + (y_geo2 - y_rock2)**2)
geo2_dists.append(dist_geo2_rock2)
dist_geo3_rock2 = np.sqrt((x_geo3 - x_rock2)**2 + (y_geo3 - y_rock2)**2)
geo3_dists.append(dist_geo3_rock2)
dist_geo4_rock2 = np.sqrt((x_geo4 - x_rock2)**2 + (y_geo4 - y_rock2)**2)
geo4_dists.append(dist_geo4_rock2)

# Geophone 4 Rock 3
sources.append('GEO 4 rock 3')
dist_geo1_rock3 = np.sqrt((x_geo1 - x_rock3)**2 + (y_geo1 - y_rock3)**2)
geo1_dists.append(dist_geo1_rock3)
dist_geo2_rock3 = np.sqrt((x_geo2 - x_rock3)**2 + (y_geo2 - y_rock3)**2)
geo2_dists.append(dist_geo2_rock3)
dist_geo3_rock3 = np.sqrt((x_geo3 - x_rock3)**2 + (y_geo3 - y_rock3)**2)
geo3_dists.append(dist_geo3_rock3)
dist_geo4_rock3 = np.sqrt((x_geo4 - x_rock3)**2 + (y_geo4 - y_rock3)**2)
geo4_dists.append(dist_geo4_rock3)

### Save distances

In [10]:
d = {'source':sources, 'geo1_distance_m':geo1_dists, 'geo2_distance_m':geo2_dists, 'geo3_distance_m':geo3_dists, 'geo4_distance_m':geo4_dists}
df = pd.DataFrame(data = d)
df.to_csv(mqdir1 + 'source_receiver_distances.csv',index=False)

## Add source-receiver distances to catalogs

In [11]:
# LM - repeating
mqdir2 = '../catalogs/final_catalogs/LM_vs_boulder_vs_other/'
cat_LM_rpt = pd.read_csv(mqdir2 + 'A17_repeating_LM_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_LM_rpt)):
    row = cat_LM_rpt.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_LM)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_LM)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_LM)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_LM)
cat_LM_rpt['source_distance_m'] = source_distances
cat_LM_rpt.to_csv(mqdir2 + 'A17_repeating_LM_catalog_HQ_final.csv',index=False)

In [12]:
# LM - isolated
cat_LM_isol = pd.read_csv(mqdir2 + 'A17_isolated_LM_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_LM_isol)):
    row = cat_LM_isol.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_LM)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_LM)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_LM)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_LM)
cat_LM_isol['source_distance_m'] = source_distances
cat_LM_isol.to_csv(mqdir2 + 'A17_isolated_LM_catalog_HQ_final.csv',index=False)

In [13]:
# LM - all
cat_LM = pd.read_csv(mqdir2 + 'A17_LM_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_LM)):
    row = cat_LM.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_LM)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_LM)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_LM)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_LM)
cat_LM['source_distance_m'] = source_distances
cat_LM.to_csv(mqdir2 + 'A17_LM_catalog_HQ_final.csv',index=False)

In [14]:
# Geophone Rock
cat_georock = pd.read_csv(mqdir2 + 'GeophoneRock_events_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_georock)):
    row = cat_georock.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_georock)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_georock)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_georock)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_georock)
cat_georock['source_distance_m'] = source_distances
cat_georock.to_csv(mqdir2 + 'GeophoneRock_events_catalog_HQ_final.csv',index=False)

In [15]:
# R2
cat_R2 = pd.read_csv(mqdir2 + 'R2Rock_events_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_R2)):
    row = cat_R2.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_R2)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_R2)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_R2)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_R2)
cat_R2['source_distance_m'] = source_distances
cat_R2.to_csv(mqdir2 + 'R2Rock_events_catalog_HQ_final.csv',index=False)

In [16]:
# Geophone 4 Rock 1
cat_rock1 = pd.read_csv(mqdir2 + 'GEO4Rock1_events_catalog_HQ_final.csv')
source_distances = []
for r in np.arange(0,len(cat_rock1)):
    row = cat_rock1.iloc[r]
    if row.geophone == 1:
        source_distances.append(dist_geo1_rock1)
    elif row.geophone == 2:
        source_distances.append(dist_geo2_rock1)
    elif row.geophone == 3:
        source_distances.append(dist_geo3_rock1)
    elif row.geophone == 4:
        source_distances.append(dist_geo4_rock1)
cat_rock1['source_distance_m'] = source_distances
cat_rock1.to_csv(mqdir2 + 'GEO4Rock1_events_catalog_HQ_final.csv',index=False)

In [17]:
# Geophone 4 Rock 2/3
cat_rock23 = pd.read_csv(mqdir2 + 'GEO4Rock2or3_events_catalog_HQ_final.csv')
distances_rock2 = []
distances_rock3 = []
for r in np.arange(0,len(cat_rock23)):
    row = cat_rock23.iloc[r]
    if row.geophone == 1:
        distances_rock2.append(dist_geo1_rock2)
        distances_rock3.append(dist_geo1_rock3)
    elif row.geophone == 2:
        distances_rock2.append(dist_geo2_rock2)
        distances_rock3.append(dist_geo2_rock3)
    elif row.geophone == 3:
        distances_rock2.append(dist_geo3_rock2)
        distances_rock3.append(dist_geo3_rock3)
    elif row.geophone == 4:
        distances_rock2.append(dist_geo4_rock2)
        distances_rock3.append(dist_geo4_rock3)
        
cat_rock23['rock2_distance_m'] = distances_rock2
cat_rock23['rock3_distance_m'] = distances_rock3
cat_rock23.to_csv(mqdir2 + 'GEO4Rock2or3_events_catalog_HQ_final.csv',index=False)